In [5]:
import sys
sys.path.append("src/")
sys.path.append("../src/")
from utils.utils import LocalPLM, LocalModelArguments
from utils import preprocess as pr

In [18]:
data, _ = pr.load_dataset("data_combined.csv", "content", "label", ratio=0.1)

Casting to class labels:   0%|          | 0/39132 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3913 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/3913 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/3913 [00:00<?, ? examples/s]

In [23]:
args = LocalModelArguments(
    model_name_or_path = "microsoft/Phi-4-mini-instruct",
    cuda_devices = "0",
    use_4bit_quantization = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = "float16",
    use_nested_quant = True,
    use_reentrant = True
)

model = LocalPLM(args)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
FINETUNED_LLM_PATH = "models/Test"

LORA_RANK_DIMENSION = 6 # the rank of the adapter, the lower the fewer parameters you'll need to train. (smaller = more compression)
LORA_ALPHA = 8 # this is the scaling factor for LoRA layers (higher = stronger adaptation)
LORA_DROPOUT = 0.05 # dropout probability for LoRA layers (helps prevent overfitting)
MAX_SEQ_LENGTH = 64
EPOCHS=1
LEARNING_RATE=2e-4

In [32]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=LORA_RANK_DIMENSION,
    lora_alpha=LORA_ALPHA,
    bias="none",
    lora_dropout=LORA_DROPOUT,
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [33]:
from trl import SFTConfig

sft_config = SFTConfig(
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    gradient_accumulation_steps=1,
    per_device_train_batch_size=16,
    auto_find_batch_size=True,
    
    max_seq_length=MAX_SEQ_LENGTH,
    packing=True,
    
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    optim='adamw_torch_fused',
    warmup_ratio=0.03,
    lr_scheduler_type="constant", 
    
    logging_steps=10,
    logging_dir='./logs',
    output_dir=FINETUNED_LLM_PATH,
    report_to='none'
)

In [ ]:
trainer, history = model.finetune(
    train_dataset=data,
    lora_config=lora_config, sft_config=sft_config
)

Converting train dataset to ChatML:   0%|          | 0/3913 [00:00<?, ? examples/s]

Parameter 'fn_kwargs'={'tokenizer': GPT2TokenizerFast(name_or_path='microsoft/Phi-4-mini-instruct', vocab_size=200019, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	199999: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200018: AddedToken("<|endofprompt|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200019: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	200020: AddedToken("<|end|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	200021: AddedToken("<|user|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	200022: AddedToken("<|sys

Applying chat template to train dataset:   0%|          | 0/3913 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3913 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/3913 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
